In [1]:
!pip install transformers==4.3.0

In [2]:
!pip install sentencepiece

In [3]:
!pip install accelerate

In [4]:
!pip install  xlrd==1.1.0

In [5]:
!pip install --upgrade ipywidgets

In [ ]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

class Predictor_FLAN_T5(Predictor):
    def __init__(self, name: str, 
    model_path: str=None,
    model_online_path: str=None,
    description: str='',
    model_type: str=None) -> None:
        
    def predict_essay(self, essay : str) -> Dict[str,str]:
        # Load data into pandas DataFrame
        data_df = pd.read_excel("StudentEssays.xlsx")

        # Initialize T5 tokenizer and model
        tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
        model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

        # Move the model to the CUDA device if available
        if torch.cuda.is_available():
            model.to("cuda")

        # Define a list of concepts to predict
        concepts_to_predict = ["potential energy", "kinetic energy", "Law of Conservation of Energy"]

        # Define possible outcome labels
        outcome_labels = ["Acceptable", "Unacceptable", "Insufficient", "Not Found"]

        # Create a list to store predictions as dictionaries
        predictions_list = []

        text = essay  # Assuming the text content is in column 'Essay'

        # Initialize predictions dictionary for this row
        predictions = {}

        # Iterate through each concept to predict
        for concept in concepts_to_predict:
            # Define a template for classification
            template = f"According to the following essay, is the student's definition of {concept} Acceptable, Unacceptable, Insufficient, or Not Found? Only use one of these labels for outputs\n{text}"
            # Prepare the input by replacing placeholders
            formatted_input = template
            # Tokenize and classify the text
            input_ids = tokenizer(formatted_input, return_tensors="pt", padding=True, truncation=True).input_ids.to("cuda" if torch.cuda.is_available() else "cpu")
            outputs = model.generate(input_ids, max_length=128)
            decoded_output = tokenizer.decode(outputs[0], skip_special_tokens=True)  # Remove special tokens

            # Store the prediction in the dictionary
            predictions[concept] = next((label for label in outcome_labels if label.lower() in decoded_output.lower()), "Unknown")

            if predictions[concept] == "Unknown":
              print(len(decoded_output))
              with open('output.txt', 'w') as f:
                f.write(decoded_output)

        return predictions